In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import scipy as sp
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [2]:
path_df = "student-mat.csv"

df = pd.read_csv(path_df)

df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


Feature Engineering

In [17]:
col_tf = ColumnTransformer(
    transformers=[
        ('OneHot', OneHotEncoder(drop="first", categories='auto'), ['Mjob','Fjob','reason','guardian']),
        ('Ordinal',OrdinalEncoder(),['school','sex','address','famsize','Pstatus','Medu','Fedu','studytime','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic']),
        ('MinMax',MinMaxScaler(),['age','traveltime','studytime','failures','famrel','freetime','goout','absences','G1','G2','G3'])
    ],
    remainder='passthrough'
)

col_tf.fit(df)
new_df = pd.DataFrame(col_tf.transform(df), columns=all_column_names)
new_df["age"]

0      0.428571
1      0.285714
2      0.000000
3      0.000000
4      0.142857
         ...   
390    0.714286
391    0.285714
392    0.857143
393    0.428571
394    0.571429
Name: age, Length: 395, dtype: float64

Now it's time to split the train and test

In [43]:
X = new_df.iloc[:, :-3]
y = new_df.iloc[:,-3:]
yDalc, yWalc, yHealth = y[["Dalc"]], y[["Walc"]], y[["health"]]
X_train, X_test, yHealth_train, yHealth_test = train_test_split(X, yHealth, random_state=42, stratify=yHealth)